# Проект: Рекомендация тарифов

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Открытие-и-изучение-данных" data-toc-modified-id="Открытие-и-изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Открытие и изучение данных</a></span></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#Исследования-моделей" data-toc-modified-id="Исследования-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследования моделей</a></span><ul class="toc-item"><li><span><a href="#Модель:-решающее-дерево" data-toc-modified-id="Модель:-решающее-дерево-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель: решающее дерево</a></span></li><li><span><a href="#Модель:-случайный-лес" data-toc-modified-id="Модель:-случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель: случайный лес</a></span></li><li><span><a href="#Модель:-логистическая-регрессия" data-toc-modified-id="Модель:-логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель: логистическая регрессия</a></span></li></ul></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span><ul class="toc-item"><li><span><a href="#Тест-модели-решающее-дерево" data-toc-modified-id="Тест-модели-решающее-дерево-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Тест модели решающее дерево</a></span></li><li><span><a href="#Тест-модели-случайный-лес" data-toc-modified-id="Тест-модели-случайный-лес-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Тест модели случайный лес</a></span></li><li><span><a href="#Тест-модели-логистическая-регрессия" data-toc-modified-id="Тест-модели-логистическая-регрессия-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Тест модели логистическая регрессия</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-адекватность" data-toc-modified-id="Проверка-лучшей-модели-на-адекватность-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Проверка лучшей модели на адекватность</a></span></li></ul></div>

## Открытие и изучение данных

Загрузим все библиотеки, которые нам понадобятся и откроем датасет.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные в порядке, пропусков нет.

Ещё посмотрим на корреляции между признаками:

In [4]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Колонки со звонками и минутами сильно между собой коррелируют. Отбросим ту, которая менее коррелирует с колонкой тарифа:

In [5]:
df = df.drop(['minutes'], axis=1)

In [6]:
df.head()

,calls,messages,mb_used,is_ultra
0,40.0,83.0,19915.42,0
1,85.0,56.0,22696.96,0
2,77.0,86.0,21060.45,0
3,106.0,81.0,8437.39,1
4,66.0,1.0,14502.75,0


Теперь можно двигаться дальше.

## Разделение данных на выборки

Разделим данные на две выборки: для обучения и целевую.

In [7]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Сперва отделим от выборок тестовые:

In [8]:
features, features_test, target, target_test = train_test_split(
    features, target, test_size=.2, random_state=123)

Теперь основные выборки разделим на тренировочную и валидационную:

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=.25, random_state=123)

Теперь можно проводить исследования.

## Исследования моделей

### Модель: решающее дерево

Сперва возьмём модель решающего дерева обучим на тренировочной выборке.<br>
Сделаем цикл, в котором переберём разную глубину дерева, выберем самую удачную и сохраним модель.
Измерим скорость обучения и подбора лучшей модели. Для этого воспользуемся мэджиком "time":

In [10]:
%%time
best_tree_model = None
best_tree_result = 0
best_tree_depth = 0
for depth in range(1, 21):
    model_dtc = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model_dtc.fit(features_train, target_train)
    predictions_dtc = model_dtc.predict(features_valid)
    result_dtc = accuracy_score(target_valid, predictions_dtc)
    if result_dtc > best_tree_result:
        best_tree_model = model_dtc
        best_tree_result = result_dtc
        best_tree_depth = depth
print('Лучшая глубина модели:', best_tree_depth)
print('Лучший результат решающего дерева:', best_tree_result.round(2))

Лучшая глубина модели: 7
Лучший результат решающего дерева: 0.79
CPU times: user 154 ms, sys: 3.25 ms, total: 157 ms
Wall time: 163 ms


Получилось, что самая удачная глубина решающего дерева: 7. На этой глубине результат: 0.79. Скорость получилась довольно высокая: около 150 милисекунд.

### Модель: случайный лес

Теперь обучим модель случайный лес и посмотрим на качество обучения. <br>
Тоже сделаем цикл, в котором переберём разное количество деревьев ,выберем наилучший вариант и измерим скорость выполнения:

In [11]:
%%time
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
for est in range(1, 26):
    model_rfc = RandomForestClassifier(random_state=123, n_estimators=est)
    model_rfc.fit(features_train, target_train)
    result_rfc = model_rfc.score(features_valid, target_valid)
    if result_rfc > best_forest_result:
        best_forest_model = model_rfc
        best_forest_result = result_rfc
        best_forest_est = est
print('Лучшее количество деревьев:', best_forest_est)
print('Лучший результат случайного леса:', best_forest_result.round(2))

Лучшее количество деревьев: 22
Лучший результат случайного леса: 0.8
CPU times: user 1.33 s, sys: 10.1 ms, total: 1.34 s
Wall time: 1.35 s


Наиболее удачным количеством деревьев оказалось: 22. И результат получше чем у решающего дерева: 0.8. Но скорость гораздо медленнее: 1.3 секунды.

Можно ещё усложнить исследование и перебрать глубину деревьев леса. Это определённо уменьшит скорость, но возможно улучшит качество:

In [12]:
%%time
best_forest_model = None
best_forest_result = 0
best_forest_est = 0
best_forest_depth = 0

for est in range(1, 21):
    for depth in range(1, 11):
        model_rfc = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model_rfc.fit(features_train, target_train)
        result_rfc = model_rfc.score(features_valid, target_valid)
        if result_rfc > best_forest_result:
            best_forest_model = model_rfc
            best_forest_result = result_rfc
            best_forest_est = est
            best_forest_depth = depth
print('Лучшее количество деревьев:', best_forest_est)
print('Лучшая глубина:', best_forest_depth)
print('Лучший результат случайного леса:', best_forest_result.round(2))

Лучшее количество деревьев: 9
Лучшая глубина: 6
Лучший результат случайного леса: 0.81
CPU times: user 6.19 s, sys: 38.7 ms, total: 6.23 s
Wall time: 6.44 s


Качество улучшилось, но не кардинально, результат: 0.81. Но скорость очень медленная: больше 6 секунд.

### Модель: логистическая регрессия

Далее обучим и посмотрим на качество логистической регрессии, тоже проверим скорость:

In [13]:
%%time
model_lr = LogisticRegression(random_state=123)
model_lr.fit(features_train, target_train)
result_lr = model_lr.score(features_valid, target_valid)
print('Результат логистической регрессии:', result_lr.round(2))

Результат логистической регрессии: 0.71
CPU times: user 15.7 ms, sys: 169 µs, total: 15.9 ms
Wall time: 21.8 ms


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Скорость выполнения самая высокая: 15 милисекунд. Но результат логистической регрессии хуже чем у решающего дерева и случайного леса: 0.71

## Проверка моделей на тестовой выборке

### Тест модели решающее дерево

Теперь протестируем лучшие модели, которые у нас получились на тестовой выборке, которую мы отделили ранее.<br>
Заодно проверим с какой скоростью каждая из моделей справляется с решением.<br>
Сперва протестируем модель решающее дерево:

In [14]:
%%time
test_prediction_dtc = best_tree_model.predict(features_test)
test_result_dtc = accuracy_score(target_test, test_prediction_dtc)
print('Accuracy модели решающее дерево:', test_result_dtc.round(2))

Accuracy модели решающее дерево: 0.79
CPU times: user 4.2 ms, sys: 195 µs, total: 4.39 ms
Wall time: 3.07 ms


Очень хороший результат на тестовой выборке: 0.79. Скорость выполнения около 5 милисекунд.

### Тест модели случайный лес

Далее протестируем модель случайный лес. Тоже сразу проверим скорость выполнения:

In [15]:
%%time
test_result_rfc = best_forest_model.score(features_test, target_test)
print('Accuracy модели случайный лес:', test_result_rfc.round(2))

Accuracy модели случайный лес: 0.81
CPU times: user 6.85 ms, sys: 42 µs, total: 6.89 ms
Wall time: 5.88 ms


Качество на тестовой выборке чуть лучше чем у решающего дерава: 0.81. Но скорость чуть медленнее: 9 милисекунд.

### Тест модели логистическая регрессия

Теперь протестируем логистическую регрессию и проверим скорость:

In [16]:
%%time
test_result_lr = model_lr.score(features_test, target_test)
print('Accuracy логистической регресии:', test_result_lr.round(2))

Accuracy логистической регресии: 0.71
CPU times: user 4.11 ms, sys: 5 µs, total: 4.12 ms
Wall time: 2.74 ms


Скорость самая высокая: 4.5 милисекунд. Но результат хуже чем у остальных: 0.71.

## Выводы

Таким образом получается, что для текущих данных, самая удачная модель - решающее дерево. У неё довольно высокая скорость обучения и скорость принятия решения на тестовой выборке, а так-же очень хорошие результаты как на валидационной выборке: accuracy-0.79, так и на тестовой: accuracy-0.79.

Ещё можно отметить, что модель случайный лес тоже показал очень хорошие результаты: accuracy-0.81 на валидационной выборке и accuracy-0.81 на тестовой. Но время обучения у неё заняло намного больше времени чем у решающего дерева: 6.5 секунд против 150 миллисекунд решающего дерева.

## Проверка лучшей модели на адекватность

Проверим лучшую модель на адекватность:

In [17]:
dummy_clf = DummyClassifier(random_state=123)
dummy_clf.fit(features_test, target_test)

predictions_dc = dummy_clf.predict(features_test)

result_dc = dummy_clf.score(features_test, target_test)

print('Результат проверки на адекватность:', result_dc.round(2))
print('Accuracy модели решающее дерево:', test_result_dtc.round(2))

Результат проверки на адекватность: 0.59
Accuracy модели решающее дерево: 0.79


Результат модели выше, проверка пройдена успешно.